<a href="https://colab.research.google.com/github/AdaZst/DS-2002/blob/main/Finance_API_Homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Submission Code

Task 1: Fetch Basic Stock Data 10pts
- Prompt the user to input two or more stock symbols.
- Use the API to fetch data for the given symbols.
- Extract and display only the Stock Ticker, Company Name, and Current Market Price

In [ ]:
import pandas as pd
import requests

# Define the API Key
API_KEY = "[FILL WITH API KEY]"
BASE_URL = "https://yfapi.net/v6/finance/quote"

# Prompt user to enter stock symbols
symbols = input("Enter two or more stock symbols separated by commas: ").upper().split(",")

# Store results
stock_data = []

# Fetch data for each stock symbol
for symbol in symbols:
    symbol = symbol.strip()  # Remove any extra spaces
    querystring = {"symbols": symbol}
    headers = {
        'x-api-key': API_KEY
    }

    # Make the API request
    response = requests.get(BASE_URL, headers=headers, params=querystring)
    data = response.json()

    # Extract relevant information
    if "quoteResponse" in data and "result" in data["quoteResponse"]:
        stock_info = data["quoteResponse"]["result"][0]
        stock_data.append({
            "Stock Ticker": stock_info.get("symbol", "N/A"),
            "Company Name": stock_info.get("longName", "N/A"),
            "Current Market Price": f"${stock_info.get('regularMarketPrice', 'N/A'):,.2f}"
        })
    else:
        stock_data.append({
            "Stock Ticker": symbol,
            "Company Name": "N/A",
            "Current Market Price": "N/A"
        })

# Convert to DataFrame
df = pd.DataFrame(stock_data)

# Display results
print(df.head())

Task 2.1: Fetch Additional Data Using Modules 10 pts
- Allow the user to choose a module from the Quote Summary Endpoint
- Find: 52 Week High, 52 Week Low, Return on Assets (ROA)
- Conver the response into a Pandas DataFrame
- Make an API request using the chosen module
- Display the DataFrame

In [ ]:
import pandas as pd
import requests

# Define the API Key
API_KEY = "[FILL WITH API KEY]"
BASE_URL = "https://yfapi.net/v11/finance/quoteSummary/"

# Prompt user to enter stock symbol
symbol = input("Enter a stock symbol: ").upper().strip()

# Prompt user to choose a module
module = input("Enter a module (e.g., 'summaryDetail', 'financialData'): ").strip()

# Make the API request
url = f"{BASE_URL}{symbol}"
querystring = {
    "modules": module
}
headers = {
    'x-api-key': API_KEY
}

response = requests.get(url, headers=headers, params=querystring)
data = response.json()

# Extract relevant information
if "quoteSummary" in data and "result" in data["quoteSummary"]:
    result = data["quoteSummary"]["result"][0]
    module_data = result.get(module, {})

    # Extract specific fields
    extracted_data = {
        "52 Week High": module_data.get("fiftyTwoWeekHigh", {}).get("raw", "N/A"),
        "52 Week Low": module_data.get("fiftyTwoWeekLow", {}).get("raw", "N/A"),
        "Return on Assets (ROA)": module_data.get("returnOnAssets", {}).get("raw", "N/A")
    }

    # Format the extracted data (only if the value is numeric)
    formatted_data = {
        "52 Week High": f"${extracted_data['52 Week High']:,.2f}" if isinstance(extracted_data['52 Week High'], (int, float)) else "N/A",
        "52 Week Low": f"${extracted_data['52 Week Low']:,.2f}" if isinstance(extracted_data['52 Week Low'], (int, float)) else "N/A",
        "Return on Assets (ROA)": f"{extracted_data['Return on Assets (ROA)']:.2%}" if isinstance(extracted_data['Return on Assets (ROA)'], (int, float)) else "N/A"
    }

    # Convert to DataFrame
    df = pd.DataFrame([formatted_data])

    # Display results
    print(df.head())
else:
    print("No data found for the given symbol and module.")

Enter a stock symbol: AAPL
Enter a module (e.g., 'summaryDetail', 'financialData'): summaryDetail
No data found for the given symbol and module.


Task 2.2: Fetch Additional Data Using Modules 10pts
- Find Current Trending Stocks, Their Actual Name and Ticker and Current Price as well as their 52 high and low.

In [ ]:
import pandas as pd
import requests

# Define the API Key
API_KEY = "[FILL WITH API KEY]"
TRENDING_URL = "https://yfapi.net/v1/finance/trending/US"
QUOTE_URL = "https://yfapi.net/v6/finance/quote"

# Fetch trending stocks
headers = {
    'x-api-key': API_KEY
}

# Make the API request to get trending stocks
response = requests.get(TRENDING_URL, headers=headers)
data = response.json()

# Extract trending stock symbols
trending_symbols = []
if "finance" in data and "result" in data["finance"]:
    for item in data["finance"]["result"]:
        for quote in item.get("quotes", []):
            trending_symbols.append(quote.get("symbol", "N/A"))
else:
    print("No trending stocks found.")
    trending_symbols = []

# Remove duplicates and limit to top 10 trending stocks
trending_symbols = list(set(trending_symbols))[:10]

# Store results
trending_stocks = []

# Fetch detailed data for each trending stock
for symbol in trending_symbols:
    querystring = {"symbols": symbol}
    response = requests.get(QUOTE_URL, headers=headers, params=querystring)
    data = response.json()

    # Extract relevant information
    if "quoteResponse" in data and "result" in data["quoteResponse"]:
        stock_info = data["quoteResponse"]["result"][0]
        trending_stocks.append({
            "Stock Ticker": stock_info.get("symbol", "N/A"),
            "Company Name": stock_info.get("longName", "N/A"),
            "Current Market Price": f"${stock_info.get('regularMarketPrice', 'N/A'):,.2f}",
            "52 Week High": f"${stock_info.get('fiftyTwoWeekHigh', 'N/A'):,.2f}",
            "52 Week Low": f"${stock_info.get('fiftyTwoWeekLow', 'N/A'):,.2f}"
        })
    else:
        trending_stocks.append({
            "Stock Ticker": symbol,
            "Company Name": "N/A",
            "Current Market Price": "N/A",
            "52 Week High": "N/A",
            "52 Week Low": "N/A"
        })

# Convert to DataFrame
df = pd.DataFrame(trending_stocks)

# Display results
print("Trending Stocks:")
print(df.head())

Trending Stocks:
  Stock Ticker         Company Name Current Market Price 52 Week High  \
0         SNOW       Snowflake Inc.              $166.19      $194.40   
1         MARA  MARA Holdings, Inc.               $12.45       $30.71   
2         ROOT           Root, Inc.               $98.71      $141.23   
3          CRM     Salesforce, Inc.              $307.33      $369.00   
4         MRNA        Moderna, Inc.               $33.58      $170.47   

  52 Week Low  
0     $107.13  
1      $12.05  
2      $27.24  
3     $212.00  
4      $29.25  
